In [1]:
import pandas as pd
# Ladataan vain "QuantHockey"-välilehti
file_path = 'Points23-24.xlsx'
quant_hockey_data = pd.read_excel(file_path, sheet_name='QuantHockey', header=0)
# Poistetaan "Unnamed"-sarakkeet (tyhjät tai turhat sarakkeet)
quant_hockey_data = quant_hockey_data.loc[:, ~quant_hockey_data.columns.str.contains('^Unnamed')]


In [2]:
# Ladataan tiedosto uudelleen ja ohitetaan ensimmäinen rivi
quant_hockey_data = pd.read_excel(file_path, sheet_name='QuantHockey', header=1)
# Poistetaan "Unnamed"-sarakkeet, jos niitä on
quant_hockey_data = quant_hockey_data.loc[:, ~quant_hockey_data.columns.str.contains('^Unnamed')]
# Normalisoidaan sarakenimet
quant_hockey_data.columns = quant_hockey_data.columns.str.strip().str.replace(' ', '_')
# Tulostetaan ensimmäiset rivit ja sarakkeet tarkistusta varten
print(quant_hockey_data.head())
print(quant_hockey_data.columns)

   Rk              Name Team  Age Pos  GP   G    A    P  PIM  ...   G/GP  \
0   1   Nikita Kucherov  TBL   30   F  81  44  100  144   22  ...  0.543   
1   2  Nathan MacKinnon  COL   28   F  82  51   89  140   42  ...  0.622   
2   3    Connor McDavid  EDM   26   F  76  32  100  132   30  ...  0.421   
3   4    Artemi Panarin  NYR   32   F  82  49   71  120   24  ...  0.598   
4   5    David Pastrnak  BOS   27   F  82  47   63  110   47  ...  0.573   

    A/GP   P/GP SHOTS    SH%  HITS  BS  FOW  FOL     FO%  
0  1.235  1.778   306  14.4%    45  35    2    2  50.00%  
1  1.085  1.707   404  12.6%    55  69  607  707  46.19%  
2  1.316  1.737   263  12.2%   118  39  442  423  51.10%  
3  0.866  1.463   302  16.2%    18  15    1    9  10.00%  
4  0.768  1.341   382  12.3%    70  19    8   16  33.33%  

[5 rows x 50 columns]
Index(['Rk', 'Name', 'Team', 'Age', 'Pos', 'GP', 'G', 'A', 'P', 'PIM', '+/-',
       'TOI', 'ES', 'PP', 'SH', 'ESG', 'PPG', 'SHG', 'GWG', 'OTG', 'ESA',
       'PPA', 

In [3]:
import plotly.express as px
# Järjestetään pelaajat pisteiden mukaan ja valitaan top 10
top_10_players = quant_hockey_data.sort_values(by='P', ascending=False).head(10)
# Muutetaan peliaika (TOI) sekunneiksi, jos ei ole valmiiksi numeroina
if top_10_players['TOI'].dtype == 'object':
    top_10_players['TOI_seconds'] = top_10_players['TOI'].str.split(':').apply(
        lambda x: int(x[0]) * 60 + int(x[1])
    )
else:
    top_10_players['TOI_seconds'] = top_10_players['TOI']
# Muokataan sarakkeiden nimet, jos ne sisältävät ylimääräisiä välilyöntejä tai alaviivoja
top_10_players.columns = top_10_players.columns.str.strip().str.replace(' ', '_')
# Visualisointi scatter-plotilla
fig = px.scatter(
    top_10_players,
    x='G/60',  # Maalit per 60 minuuttia
    y='A/60',  # Syötöt per 60 minuuttia
    size='P/60',  # Kuplan koko pisteet per 60 minuuttia
    color='P/60',  # Väritys perustuu pisteisiin per 60
    hover_name='Name',  # Pelaajien nimet tooltipissa
    title='G/60 vs A/60 vs P/60 (Top 10 pistemiehet)',
    labels={'G/60': 'Maalit per 60 min', 'A/60': 'Syötöt per 60 min', 'P/60': 'Pisteet per 60 min'}
)
# Kuvaajan ulkoasun muokkaus
fig.update_traces(marker=dict(opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    xaxis_title="Maalit per 60 minuuttia",
    yaxis_title="Syötöt per 60 minuuttia",
    coloraxis_colorbar=dict(title="Pisteet/60")
)
fig.show()

In [4]:
import plotly.graph_objects as go

# Järjestetään pelaajat pisteiden mukaan ja valitaan top 10
top_10_players = quant_hockey_data.sort_values(by='P', ascending=False).head(10)

# Valmistellaan data
players = top_10_players['Name']
g60 = top_10_players['G/60']
a60 = top_10_players['A/60']
p60 = top_10_players['P/60']

# Rakennetaan kuvaaja
fig = go.Figure()

# Lisätään palkit
fig.add_trace(go.Bar(
    x=players,
    y=g60,
    name='Maalit per 60 min (G/60)',
    marker_color='orange'
))
fig.add_trace(go.Bar(
    x=players,
    y=a60,
    name='Syötöt per 60 min (A/60)',
    marker_color='green'
))
fig.add_trace(go.Bar(
    x=players,
    y=p60,
    name='Pisteet per 60 min (P/60)',
    marker_color='blue'
))

# Muokataan ulkoasua
fig.update_layout(
    barmode='group',
    title='G/60, A/60 ja P/60 Top 10 pistemiehille',
    xaxis_title='Pelaajat',
    yaxis_title='Arvo',
    legend_title='Tilasto',
    xaxis=dict(tickangle=-45)  # Pelaajien nimet vinoon
)

fig.show()